In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import truncnorm

# Leer DataSet

In [2]:
df = pd.read_csv("IBM_Employee_Attrition_Performance.csv")

In [3]:
df.shape

(1470, 35)

In [4]:
df.head(3)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0


### Analisis de columnas y tipos de datos

In [7]:
df.dtypes

age                          int64
attrition                   object
businessTravel              object
dailyRate                    int64
department                  object
distanceFromHome             int64
education                    int64
educationField              object
employeeCount                int64
employeeNumber               int64
environmentSatisfaction      int64
gender                      object
hourlyRate                   int64
jobInvolvement               int64
jobLevel                     int64
jobRole                     object
jobSatisfaction              int64
maritalStatus               object
monthlyIncome                int64
monthlyRate                  int64
numCompaniesWorked           int64
over18                      object
overTime                    object
percentSalaryHike            int64
performanceRating            int64
relationshipSatisfaction     int64
standardHours                int64
stockOptionLevel             int64
totalWorkingYears   

In [5]:
df.columns = ['age', 'attrition', 'businessTravel', 'dailyRate', 'department',
       'distanceFromHome', 'education', 'educationField', 'employeeCount',
       'employeeNumber', 'environmentSatisfaction', 'gender', 'hourlyRate',
       'jobInvolvement', 'jobLevel', 'jobRole', 'jobSatisfaction',
       'maritalStatus', 'monthlyIncome', 'monthlyRate', 'numCompaniesWorked',
       'over18', 'overTime', 'percentSalaryHike', 'performanceRating',
       'relationshipSatisfaction', 'standardHours', 'stockOptionLevel',
       'totalWorkingYears', 'trainingTimesLastYear', 'workLifeBalance',
       'yearsAtCompany', 'yearsInCurrentRole', 'yearsSinceLastPromotion',
       'yearsWithCurrManager']

In [6]:
df.head(3)

,age,attrition,businessTravel,dailyRate,department,distanceFromHome,education,educationField,employeeCount,employeeNumber,...,relationshipSatisfaction,standardHours,stockOptionLevel,totalWorkingYears,trainingTimesLastYear,workLifeBalance,yearsAtCompany,yearsInCurrentRole,yearsSinceLastPromotion,yearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
